In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, MaxPool2D
from keras.models import Model
from keras.models import load_model

import keras.backend as K
from keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import gc

from keras.callbacks import ReduceLROnPlateau
from PIL import Image

import keras

from keras.callbacks import Callback

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier



np.random.seed(2)


import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.utils import shuffle


from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img, save_img


Using TensorFlow backend.


## Pre-processing data / agumment data

In [2]:
# Reading our train data
train_df = pd.read_csv("./train.csv")

### Collecting top 10 whales that are not new_whale, setting to new_df

In [3]:
# collecting whales that are not new_whale
whale_df = train_df[train_df.Id != 'new_whale']
# getting top ten whales
top_ten = whale_df["Id"].value_counts().head(10)
print(top_ten)

# making a new df with top 10
columns = ['Image', 'Id']
new_df = pd.DataFrame(columns=columns)

for i in range(len(train_df['Id'])):
    if train_df['Id'].loc[i] in top_ten:
        new_df.loc[i] = (train_df['Image'].loc[i], train_df['Id'].loc[i])
new_df[:2]

w_23a388d    73
w_9b5109b    65
w_9c506f6    62
w_0369a5c    61
w_700ebb4    57
w_3de579a    54
w_564a34b    51
w_fd3e556    50
w_88e4537    49
w_2b069ba    48
Name: Id, dtype: int64


,Image,Id
26,004e8ad5b.jpg,w_3de579a
32,00570db6b.jpg,w_9c506f6


### Spliting our data set.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(new_df['Image'], new_df['Id'], test_size=0.20)

print("Shape of X_train: ", X_train.shape[0])
print("Shape of X_test: ", X_test.shape[0])
print("Shape of y_train: ", y_train.shape[0])
print("Shape of y_test: ", y_test.shape[0])
print(X_train[:2])
print(y_train[:2])

Shape of X_train:  456
Shape of X_test:  114
Shape of y_train:  456
Shape of y_test:  114
25035    fc5c296f8.jpg
19678    c669a3706.jpg
Name: Image, dtype: object
25035    w_0369a5c
19678    w_700ebb4
Name: Id, dtype: object


Our image data generator ( agumentation )

In [ ]:
gen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.15,
                        zoom_range=0.1, channel_shift_range=10,
                        horizontal_flip=True)


Processing every image from data set and resizing / agumentating

In [ ]:

def processImages(data, dataset, imageCount = 0):
    i = 0
    for fig in data['Image']:
        img = image.load_img("./input/"+dataset+"/"+fig, target_size=(100, 100, 3))
        img_arr = image.img_to_array(img)
        image.save_img('./processed/100_100_0_'+fig, img_arr)
        temp_image = np.expand_dims(mpimg.imread('./processed/100_100_0_'+fig), 0)
        aug_iter = gen.flow(temp_image)
        aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(imageCount)]
        for x in range(len(aug_images)):
            image.save_img('./processed/100_100_'+str(x+1)+'_'+fig, aug_images[x])
        if (i%100 == 0):
            print("Processing image: ", i+1, ", ", fig)
        i += 1
        

In [ ]:
processImages(new_df, 'train', 10)

### Function to prep our images to be trained.

In [5]:
def prepareImages(data, m, type_of_data, imageCount = 1):
    
    print("Preparing images")
    
    if type_of_data == 'training':
        X = np.zeros((m*imageCount, 100, 100, 3))
        count = 0
        for fig in data:
            #load images into images of size 100x100x3
            for i in range(imageCount):
                img = image.load_img('./processed/100_100_' + str(i) + '_' + fig, target_size=(100, 100, 3))
                x = image.img_to_array(img)
                x = preprocess_input(x)
                X[count] = x
                count += 1
                if (count%100 == 0):
                    print("Processing image: ", count+1, ", ", fig)
    elif type_of_data == 'testing':
        X = np.zeros((m, 100, 100, 3))
        count = 0
        for fig in data:
            img = image.load_img('./processed/100_100_0_' + fig, target_size=(100, 100, 3))
            x = image.img_to_array(img)
            x = preprocess_input(x)
            X[count] = x
            count += 1
            if (count%50 == 0):
                print("Processing image: ", count+1, ", ", fig)    
    return X

def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # print(onehot_encoded)

    y = onehot_encoded
    # print(y.shape)
    return y, label_encoder

In [6]:
X_train_processed = prepareImages(X_train, X_train.shape[0], 'training')
X_test_processed = prepareImages(X_test, X_test.shape[0], 'testing')
X_train_processed /= 255
X_test_processed /= 255

# X = prepareImages(new_df['Image'], new_df['Image'].shape[0], 'training')
# X /= 255

Preparing images
Processing image:  101 ,  e6bfde781.jpg
Processing image:  201 ,  317a126f4.jpg
Processing image:  301 ,  471e75940.jpg
Processing image:  401 ,  1eef410a0.jpg
Preparing images
Processing image:  51 ,  eaa3332d8.jpg
Processing image:  101 ,  a6c4cab26.jpg


In [7]:
print('X_train Image count: ', X_train_processed.shape)
print('X_test Image count: ', X_test_processed.shape)
# print('X_test Image count: ', X.shape)

X_train Image count:  (456, 100, 100, 3)
X_test Image count:  (114, 100, 100, 3)


In [8]:
imageCount = 1
# Setting up our y.
new_y_array = []
for x in y_train:
    for y in range(imageCount):
        new_y_array.append(x)

y_train_processed, label_encorder = prepare_labels(new_y_array)
y_test_processed, label_encorder = prepare_labels(y_test)

# y, label_encorder = prepare_labels(new_y_array)

c:\users\acons\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
c:\users\acons\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this wa

In [9]:
print('Y train processed shape: ', y_train_processed.shape)
print('Y test processed shape: ', y_test_processed.shape)
# print("Y shape: ", y.shape)

Y train processed shape:  (456, 10)
Y test processed shape:  (114, 10)


Spliting and randominizing our data for training.

In [10]:
X_train_p_n, X_test_p_n, y_train_p_n, y_test_p_n = train_test_split(X_train_processed, y_train_processed, random_state = 20, test_size=0, )

# Constructing our model

In [11]:
def cnn_model():
    keras.backend.clear_session()

    model = Sequential()

    model.add(Convolution2D(32, (3, 3), strides=(1,1), activation='relu', input_shape = (100, 100, 3)))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(64, (3, 3), strides=(1,1), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(128, (3, 3), strides=(1,1), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))


    model.add(Flatten())
    model.add(Dense(650, activation="relu"))
    model.add(Dropout(0.8))
    model.add(Dense(10, activation='softmax'))
    return model

In [12]:
model = cnn_model()

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)  

### Making a callback for testing purposes

In [13]:
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [14]:
history = model.fit(X_train_p_n, y_train_p_n, batch_size=64, epochs=40, callbacks=[TestCallback((X_test_processed, y_test_processed))], verbose = 1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/40
456/456 [==============================] - 6s 12ms/step - loss: 2.4752 - acc: 0.1009: 6s - loss: 2.6241 - acc: 0.

Testing loss: 2.29959287559777, acc: 0.13157894762984493

Epoch 2/40
456/456 [==============================] - 1s 1ms/step - loss: 2.2903 - acc: 0.1360

Testing loss: 2.294657853611729, acc: 0.1403508792843735

Epoch 3/40
456/456 [==============================] - 1s 1ms/step - loss: 2.3028 - acc: 0.1250

Testing loss: 2.2830817490293267, acc: 0.24561403613341481

Epoch 4/40
456/456 [==============================] - 1s 1ms/step - loss: 2.2579 - acc: 0.1425

Testing loss: 2.2345504593430903, acc: 0.21052631578947367

Epoch 5/40
456/456 [==============================] - 1s 2ms/step - loss: 2.1771 - acc: 0.2083

Testing loss: 2.1036037353047154, acc: 0.31578947472990604

Epoch 6/40
456/456 [==============================] - 1s 2ms/step - loss: 2.0607 - acc: 0.2281

Testing loss: 2.0027587957549513, acc: 0.385964913

In [15]:
score = model.evaluate(X_test_processed, y_test_processed, verbose=1)
print('The accuracy is :', score[1])

114/114 [==============================] - 0s 763us/step
The accuracy is : 0.6929824540489599


### Adding more agumented images to our data set.

In [16]:
X_train_processed = prepareImages(X_train, X_train.shape[0], 'training', 7)
X_test_processed = prepareImages(X_test, X_test.shape[0], 'testing', 7)
X_train_processed /= 255
X_test_processed /= 255

Preparing images
Processing image:  101 ,  ed77166d6.jpg
Processing image:  201 ,  fe7a2b654.jpg
Processing image:  301 ,  614836cf8.jpg
Processing image:  401 ,  511834bc3.jpg
Processing image:  501 ,  c64cabd50.jpg
Processing image:  601 ,  4b6831b73.jpg
Processing image:  701 ,  e6bfde781.jpg
Processing image:  801 ,  9e924e18d.jpg
Processing image:  901 ,  680070ef1.jpg
Processing image:  1001 ,  396b9f681.jpg
Processing image:  1101 ,  81b79b1fd.jpg
Processing image:  1201 ,  da3529684.jpg
Processing image:  1301 ,  877e3feda.jpg
Processing image:  1401 ,  317a126f4.jpg
Processing image:  1501 ,  931a21ac9.jpg
Processing image:  1601 ,  2d83b5312.jpg
Processing image:  1701 ,  4b3abf879.jpg
Processing image:  1801 ,  7fb338aa0.jpg
Processing image:  1901 ,  bcd3b6249.jpg
Processing image:  2001 ,  8cfcac09c.jpg
Processing image:  2101 ,  471e75940.jpg
Processing image:  2201 ,  83d05e445.jpg
Processing image:  2301 ,  fddc28401.jpg
Processing image:  2401 ,  cdb4eb77e.jpg
Processi

In [17]:
print('X_train Image count: ', X_train_processed.shape)
print('X_test Image count: ', X_test_processed.shape)

X_train Image count:  (3192, 100, 100, 3)
X_test Image count:  (114, 100, 100, 3)


In [18]:
imageCount = 7
# Setting up our y.
new_y_array = []
for x in y_train:
    for y in range(imageCount):
        new_y_array.append(x)

y_train_processed, label_encorder = prepare_labels(new_y_array)
y_test_processed, label_encorder = prepare_labels(y_test)

c:\users\acons\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
c:\users\acons\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this wa

In [19]:
print('Y train processed shape: ', y_train_processed.shape)
print('Y test processed shape: ', y_test_processed.shape)
# print("Y shape: ", y.shape)

Y train processed shape:  (3192, 10)
Y test processed shape:  (114, 10)


In [20]:
X_train_p_n, X_test_p_n, y_train_p_n, y_test_p_n = train_test_split(X_train_processed, y_train_processed, test_size=0)


In [21]:
model = cnn_model()

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 128)       73856     
__________

In [22]:
history = model.fit(X_train_p_n, y_train_p_n, batch_size=64, epochs=20, callbacks=[TestCallback((X_test_processed, y_test_processed))], verbose = 1)

Epoch 1/20
3192/3192 [==============================] - 6s 2ms/step - loss: 2.2190 - acc: 0.1839

Testing loss: 2.002661290921663, acc: 0.27192982560709905

Epoch 2/20
3192/3192 [==============================] - 5s 1ms/step - loss: 1.9059 - acc: 0.3139

Testing loss: 1.7088563860508434, acc: 0.4736842147090979

Epoch 3/20
3192/3192 [==============================] - 5s 1ms/step - loss: 1.6655 - acc: 0.3941

Testing loss: 1.3932089429152639, acc: 0.5877192971999186

Epoch 4/20
3192/3192 [==============================] - 5s 1ms/step - loss: 1.4240 - acc: 0.4809

Testing loss: 1.272735361467328, acc: 0.5438596480771115

Epoch 5/20
3192/3192 [==============================] - 5s 1ms/step - loss: 1.2253 - acc: 0.5498

Testing loss: 1.0306818955822994, acc: 0.6491228049261528

Epoch 6/20
3192/3192 [==============================] - 5s 1ms/step - loss: 1.0620 - acc: 0.6150

Testing loss: 0.9175939110287449, acc: 0.7105263126523871

Epoch 7/20
3192/3192 [==============================] - 5s 

In [23]:
score = model.evaluate(X_test_processed, y_test_processed, verbose=1)
print('The accuracy is :', score[1])

114/114 [==============================] - 0s 789us/step
The accuracy is : 0.8333333301962468
